#### load libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV, ElasticNet, ElasticNetCV, HuberRegressor
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import cross_val_score, cross_val_predict,cross_validate, GridSearchCV, train_test_split
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor 
import statistics
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.max_columns = None
pd.options.display.max_rows = None
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#### load and inspect data

In [254]:
ames_df = pd.read_csv('./data/final_df.csv', index_col = 0)

# Checking final_df after generating geoCode, found duplicated observations
ames_df.shape

(2603, 179)

In [255]:
# drop duplicated observations
ames_df = ames_df.drop_duplicates(subset=['PID'],keep = 'first')
ames_df.shape

(2558, 179)

#### preprocessing

In [256]:
ames_df_use = ames_df.copy()

# convert MSSubClass to str
ames_df_use['MSSubClass'] = ames_df_use[['MSSubClass']].astype('str')

# characterize tax distr and sch distr
ames_df_use['TxD_S'] = ames_df_use[['TxD_S']].astype('str')
ames_df_use['SchD_S'] = ames_df_use[['SchD_S']].astype('str')

# convert NA's to 0s in LotFrontage and Alley
ames_df_use['LotFrontage'] = np.where(pd.isnull(ames_df_use['LotFrontage']), 0, ames_df_use['LotFrontage'])
ames_df_use['Alley'] = np.where(pd.isnull(ames_df_use['Alley']), 0, 1)

# impute None to Nan value in columns
none_features = ['FireplaceQu','GarageType','GarageFinish','GarageQual','GarageCond','Fence','BsmtQual','BsmtCond']
for feature in none_features:
    ames_df_use[feature] = ames_df_use[feature].fillna('None')

# impute None to 0 values in columns
zero_features = ['1stFlrSF','2ndFlrSF','GarageCars','TtlBsmtSF','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','GarageArea',
                 'BsmtFullBath','BsmtHalfBath','FullBath','HalfBath','Fireplaces']
for feature in zero_features:
    ames_df_use[feature] = ames_df_use[feature].fillna(0)

# drop rows with NA garage car / garage area
garage_NA_index = ames_df_use.loc[pd.isnull(ames_df_use['GarageArea'])].index
ames_df_use = ames_df_use.drop(garage_NA_index,axis=0)

# add in proper bathroom numbers for 918 and 2328
ames_df_use.at[918, 'num_bathroom'] = 1
ames_df_use.at[2328, 'num_bathroom'] = 3.5

#### descriptive modeling

In [257]:
X_full = ames_df_use[['YearBuilt','GrLivArea','OverallQual','OverallCond','TtlBsmtSF','BsmtUnfSF',\
                      'GarageArea','GarageQual','KitchenAbvGr','KitchenQual','BedroomAbvGr','FullBath',\
                      'Fireplaces','FireplaceQu','Neighborhood','MSSubClass','TxD_S']]

# X_full['1stFlrSF'] = np.log(X_full['1stFlrSF']+1)
# X_full['2ndFlrSF'] = np.log(X_full['2ndFlrSF']+1)
X_full['GrLivArea'] = np.log(X_full['GrLivArea'])
X_full['TtlBsmtSF'] = np.log(X_full['TtlBsmtSF']+1)
# X_full['BsmtFinSF1'] = np.log(X_full['BsmtFinSF1']+1)
# X_full['BsmtFinSF2'] = np.log(X_full['BsmtFinSF2']+1)
X_full['GarageArea'] = np.log(X_full['GarageArea']+1)

X_full_dum = pd.get_dummies(X_full, drop_first = True)
print(X_full_dum.shape)

y_full = np.log(ames_df_use['SalePrice'])

(2558, 66)


C:\Users\aznan\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\aznan\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\aznan\anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.p

In [258]:
lin_reg = LinearRegression(normalize = True).fit(X_full_dum, y_full)
lin_reg.score(X_full_dum, y_full)

x_feature = sm.add_constant(X_full_dum)

model = sm.OLS(y_full, x_feature)
results_feature = model.fit()
print(results_feature.summary())
pValue = results_feature.pvalues
pValue[pValue<0.05]

0.9161352074101354

                            OLS Regression Results                            
Dep. Variable:              SalePrice   R-squared:                       0.916
Model:                            OLS   Adj. R-squared:                  0.914
Method:                 Least Squares   F-statistic:                     418.8
Date:                Mon, 30 Nov 2020   Prob (F-statistic):               0.00
Time:                        20:54:28   Log-Likelihood:                 1987.4
No. Observations:                2558   AIC:                            -3843.
Df Residuals:                    2492   BIC:                            -3457.
Df Model:                          65                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                    0.6695 

YearBuilt                1.640017e-44
GrLivArea               9.338704e-226
OverallQual              1.015881e-65
OverallCond              4.982948e-97
TtlBsmtSF                4.125374e-52
BsmtUnfSF                1.665407e-31
GarageArea               9.468491e-18
KitchenAbvGr             6.422256e-06
BedroomAbvGr             1.567246e-05
Fireplaces               1.468775e-07
GarageQual_Fa            1.406765e-02
GarageQual_None          2.369733e-02
GarageQual_Po            1.419613e-03
GarageQual_TA            1.893787e-02
KitchenQual_Fa           2.844699e-09
KitchenQual_Gd           4.986836e-14
KitchenQual_TA           1.264190e-17
FireplaceQu_TA           2.016512e-02
Neighborhood_ClearCr     2.729804e-05
Neighborhood_Crawfor     3.119073e-07
Neighborhood_Greens      1.994292e-02
Neighborhood_IDOTRR      6.533642e-04
Neighborhood_MeadowV     1.957688e-02
Neighborhood_NoRidge     3.489850e-10
Neighborhood_NridgHt     1.815439e-04
Neighborhood_OldTown     2.725094e-03
Neighborhood

In [259]:
# remove unnecessary columns
ames_df_use = ames_df_use.drop(
    ['PA-PreD','PA-PostD','PA-UnTyp','PA-UntNo','X1TPr_D','X1TSc_D','Rcrd_Mo','Legal_Pr','SchD_S',\
    'X2TPr_D','X2TSc_D','X1TPr_S','X1TSc_S','X2TPr_S','X2TSc_S','ISU_lat_long','address','MA_Ownr1','MA_Ownr2',\
    'MA_Line1','MA_Line2','MA_City','MA_State','address3','location2','point2','SaleCond','Source','Date',\
    'ParType','BldgNo_S','DwlgNo_S','YrBuilt','Ext1','Ext2','GLA','GarYrBlt','Cars','MA_Zip1','MA_Zip2',\
    'ZngCdPr','ZngCdSc','ZngOLPr','ZngOLSc','PA-Nmbr','PA-Strt','PA-StSfx','Inst1_No','Inst1_Yr','Inst1_Mo',\
    'Inst1TPr','TtlVal_AsrYr','ValType','OthAc_S','ImpAc_S','LndAc_S','Prop_Addr','HSTtl_D','MilVal_D',\
    'HSTtl_S','MilVal_S','GeoRefNo','Tier','Range','AcreX_S1','ClassPr_S','ClassSc_S','LndAcX1S','ImpAcX1S',\
    'Central Air','ImpAcX2S','AcreGr','AcreNt_S','ParclRel','Rcrd_Yr','address2','SaleType',\
    'latitude2','longitude2','ISU_lat','ISU_long','altitude2','Central Air',\

    'index','MiscVal','YrSold_YYYY','MoSold_MM','PoolArea','PoolQC','MiscFeature','Street','PID','Utilities',\
    'BsmtHalfBath2','FullBath','HalfBath','TtlBsmtSF','HalfBath2','BsmtFullBath','BsmtHalfBath',\
    'BsmtFinType1','BsmtFinType2','LowQualFinSF',\
    '1stFlrSF','2ndFlrSF','MasVnrType','MasVnrArea','GarageArea2','PoolArea2','NmbrBRs',\
    'TotalBsmtSF','GarageYrBlt','BasmtFinSF','Heating'],\
    axis=1)

print(ames_df_use.columns)
print(f'has null: {ames_df_use.columns[ames_df_use.isnull().sum() > 0]}')

Index(['GrLivArea', 'SalePrice', 'Distance', 'MSSubClass', 'MSZoning',
       'LotFrontage', 'LotArea', 'Alley', 'LotShape', 'LandContour',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt',
       'YearRemodAdd', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd',
       'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'HeatingQC',
       'CentralAir', 'Electrical', 'BedroomAbvGr', 'KitchenAbvGr',
       'KitchenQual', 'TotRmsAbvGrd', 'Functional', 'Fireplaces',
       'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageCars', 'GarageArea',
       'GarageQual', 'GarageCond', 'PavedDrive', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'Fence', 'MoSold',
       'YrSold', 'SaleCondition', 'MasVnrArea2', 'BasmtFinSF1', 'BasmtFinSF2',
       'total_LivArea', 'num_bathroom', 

#### feature engineering

In [260]:
def feature_eng(df):
    # num_bathroom = adding up all the bathroom variables
    
    # calculate age of building
    df['BldgAge'] = df['YrSold'] - df['YearBuilt']
    #effectiveage: df['BldgAge'] = df['YearBuilt'] - df['YearRemodAdd']
    
    # adding up BsmtFinSF 1 and 2
    df['BsmtTtlFinSF'] = df['BsmtFinSF1'] + df['BsmtFinSF2']
    
    # binarize YearRemodAdd
    df['Remodeled'] = np.where(df['YearRemodAdd'] == df['YearBuilt'], 0, 1)

    # binarize MSSubClass to PUD or not PUD
    df['IsPUD'] = np.where(df['MSSubClass'].isin(['120','150','160','180']), 1, 0)
    
    # binarize Condition1/2 to positive feature or no positive feature
    df['PosFeat'] = np.where(df['Condition1'].isin(['PosN','PosA'])|df['Condition2'].isin(['PosN','PosA']), 1, 0)
    
    # combine exterior material 1/2 to one column
    df['ExtMatl'] = np.where((df['Exterior1st']==df['Exterior2nd']),df['Exterior1st'], 'Mixed')
    
    # simply qual/cond features
    for col in ['ExterQual','BsmtQual','KitchenQual','FireplaceQu','GarageQual','ExterCond',\
                'BsmtCond','GarageCond','HeatingQC']:
        df[col+'_num'] = df[col].replace(['Ex','Gd','TA','Fa','Po','None'],[10,8,6,4,2,0])

    # sum up porch/housefront area
    df['TotalPorchSF'] = df['OpenPorchSF'] + df['3SsnPorch'] + df['EnclosedPorch'] +\
                                df['ScreenPorch'] + df['WoodDeckSF']
    
    # binarize fences to having a fence or not having a fence
    df['HasFence'] = np.where(df['Fence']=='None', 0, 1)

    # simplify Functional to 3 classes
    df['Funct_3'] = df['Functional'].replace(['Maj1', 'Maj2', 'Min1', 'Min2', 'Mod', 'Sal', 'Typ'],\
                                             ['ModToSev','ModToSev','Minor','Minor','ModToSev','ModToSev','Normal'])
    
    # assign a direction for each neighborhood
    df['Direction'] = df['Neighborhood'].replace(['Blmngtn','Blueste','BrDale','BrkSide','ClearCr','CollgCr','Crawfor',\
                                                  'Edwards','Gilbert','Greens','IDOTRR','MeadowV','Mitchel','NAmes',\
                                                  'NPkVill','NWAmes','NoRidge','NridgHt','OldTown','SWISU','Sawyer',\
                                                  'SawyerW','Somerst','StoneBr','Timber','Veenker'],\
                                                 ['N','S','N','E','N','W','S','W','N','S','W','S','S','N','N','N','N',\
                                                 'N','E','S','W','W','N','N','S','N'])
    
    # drop the original columns or unused columns
    df = df.drop(['MSSubClass','YearBuilt','YearRemodAdd','LotFrontage','LotArea','LotConfig','LandSlope',\
                  'Condition1','Condition2','Exterior1st','Exterior2nd','LotShape','LandContour',\
                  'OpenPorchSF','3SsnPorch','EnclosedPorch','ScreenPorch','WoodDeckSF',\
                  'Fence','Functional','ExterQual','BsmtQual','KitchenQual','FireplaceQu',\
                  'GarageQual','ExterCond','BsmtCond','GarageCond','HeatingQC','MSZoning',\
                  'GarageCars','RoofStyle','RoofMatl','KitchenAbvGr','Functional','Neighborhood',\
                  'BsmtFinSF1', 'BsmtFinSF2','BasmtFinSF1', 'BasmtFinSF2','BedroomAbvGr',\
                 'total_LivArea','Alley','MoSold','YrSold'],\
                 axis = 1)
#     df = df.drop(['MSSubClass','YearBuilt','YearRemodAdd','LotFrontage','LotArea','LotConfig','LandSlope',\
#                   'Condition1','Condition2','Exterior1st','Exterior2nd','LotShape','LandContour',\
#                   'OpenPorchSF','3SsnPorch','EnclosedPorch','ScreenPorch','WoodDeckSF',\
#                   'Fence','Functional','ExterQual','BsmtQual','KitchenQual','FireplaceQu',\
#                   'GarageQual','ExterCond','BsmtCond','GarageCond','HeatingQC',\
#                   'GarageCars','RoofStyle','RoofMatl','KitchenAbvGr'],\
    #,'ExterQual_num','ExterCond_num'
    return df

In [261]:
ames_df_eng = feature_eng(ames_df_use)

In [262]:
ames_df_eng.columns

Index(['GrLivArea', 'SalePrice', 'Distance', 'BldgType', 'HouseStyle',
       'OverallQual', 'OverallCond', 'Foundation', 'BsmtExposure', 'BsmtUnfSF',
       'CentralAir', 'Electrical', 'TotRmsAbvGrd', 'Fireplaces', 'GarageType',
       'GarageFinish', 'GarageArea', 'PavedDrive', 'SaleCondition',
       'MasVnrArea2', 'num_bathroom', 'TxD_S', 'BldgAge', 'BsmtTtlFinSF',
       'Remodeled', 'IsPUD', 'PosFeat', 'ExtMatl', 'ExterQual_num',
       'BsmtQual_num', 'KitchenQual_num', 'FireplaceQu_num', 'GarageQual_num',
       'ExterCond_num', 'BsmtCond_num', 'GarageCond_num', 'HeatingQC_num',
       'TotalPorchSF', 'HasFence', 'Funct_3', 'Direction'],
      dtype='object')

#### dummification and log transformation

In [263]:
# dummify the dataset
ames_df_dum = pd.get_dummies(ames_df_eng, drop_first=True)
print(ames_df_dum.shape)
print(ames_df_dum.shape)

# apply log()
ames_df_dum['SalePrice'] = np.log(ames_df_dum['SalePrice'])
ames_df_dum['GrLivArea'] = np.log(ames_df_dum['GrLivArea'])
ames_df_dum['BsmtUnfSF'] = np.log(ames_df_dum['BsmtUnfSF']+1)
ames_df_dum['BsmtTtlFinSF'] = np.log(ames_df_dum['BsmtTtlFinSF']+1)
# ames_df_dum['total_LivArea'] = np.log(ames_df_dum['total_LivArea'])
# ames_df_dum['Distance'] = np.log(X_train_dum['Distance'])
ames_df_dum['TotalPorchSF'] = np.log(ames_df_dum['TotalPorchSF']+1)

(2558, 84)
(2558, 84)


#### test-train split

In [264]:
features = ames_df_dum.drop('SalePrice', axis = 1)
target = ames_df_dum['SalePrice']

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(2046, 83)
(512, 83)
(2046,)
(512,)


#### linear regression, p-values, and VIFs

In [265]:
lin_reg = LinearRegression().fit(X_train, y_train)
lin_reg.score(X_train, y_train)

x_feature = sm.add_constant(X_train)

model = sm.OLS(y_train, x_feature)
results_feature = model.fit()
print(results_feature.summary())
pValue = results_feature.pvalues
pValue[pValue<0.05]

#X_vif = X_train_dum[pValue[pValue<0.05].index]
X_vif = X_train[pValue[pValue<0.05].drop('const').index]
vif_data = pd.DataFrame() 
vif_data["feature"] = X_vif.columns
vif_data["VIF"] = [variance_inflation_factor(X_vif.values, i) 
                          for i in range(len(X_vif.columns))] 
print(vif_data)

0.9168072510551726

                            OLS Regression Results                            
Dep. Variable:              SalePrice   R-squared:                       0.917
Model:                            OLS   Adj. R-squared:                  0.913
Method:                 Least Squares   F-statistic:                     263.8
Date:                Mon, 30 Nov 2020   Prob (F-statistic):               0.00
Time:                        20:54:39   Log-Likelihood:                 1593.7
No. Observations:                2046   AIC:                            -3021.
Df Residuals:                    1963   BIC:                            -2555.
Df Model:                          82                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     6.88

const                     0.000000e+00
GrLivArea                1.846055e-116
Distance                  9.164807e-11
OverallQual               2.075130e-52
OverallCond               1.839280e-53
Fireplaces                1.227234e-05
GarageArea                2.369191e-18
num_bathroom              1.282327e-06
BldgAge                   7.290454e-20
BsmtTtlFinSF              3.819523e-14
Remodeled                 1.912591e-02
ExterQual_num             1.094350e-03
BsmtQual_num              9.643604e-07
KitchenQual_num           1.150365e-06
HeatingQC_num             4.341507e-04
TotalPorchSF              1.502765e-02
BldgType_Duplex           1.526434e-11
BldgType_Twnhs            1.800779e-02
HouseStyle_1.5Unf         6.991112e-04
HouseStyle_1Story         1.912052e-12
HouseStyle_2Story         2.376466e-02
HouseStyle_SFoyer         7.176247e-04
Foundation_PConc          3.392929e-02
BsmtExposure_Gd           1.913596e-04
BsmtExposure_Mn           4.339350e-05
BsmtExposure_No          

                  feature         VIF
0               GrLivArea  298.484037
1                Distance   16.025057
2             OverallQual   70.582935
3             OverallCond   40.597790
4              Fireplaces    2.809241
5              GarageArea   10.299096
6            num_bathroom   23.029594
7                 BldgAge   13.242638
8            BsmtTtlFinSF    5.135345
9               Remodeled    2.873469
10          ExterQual_num  110.280083
11           BsmtQual_num   42.316412
12        KitchenQual_num   75.386694
13          HeatingQC_num   35.013638
14           TotalPorchSF    7.114611
15        BldgType_Duplex    1.382368
16         BldgType_Twnhs    1.177987
17      HouseStyle_1.5Unf    1.078427
18      HouseStyle_1Story    4.961935
19      HouseStyle_2Story    3.664722
20      HouseStyle_SFoyer    1.426759
21       Foundation_PConc    4.815561
22        BsmtExposure_Gd    1.834909
23        BsmtExposure_Mn    1.796025
24        BsmtExposure_No    6.976635
25          

In [266]:
print(f'Num of features: {len(X_train.columns)}')
print(f'AIC: {results_feature.aic}')
print(f'BIC: {results_feature.bic}')
print(f'train score: {lin_reg.score(X_train, y_train)}')
print(f'test score: {lin_reg.score(X_test, y_test)}')

Num of features: 83
AIC: -3021.370803173573
BIC: -2554.6085216131128
train score: 0.9168072510551726
test score: 0.9220260378715811


In [267]:
sig_feat = pValue[pValue<0.05].index.drop('const')

In [268]:
X_train2 = X_train[sig_feat]
X_test2 = X_test[sig_feat]

In [269]:
lin_reg = LinearRegression().fit(X_train2, y_train)
lin_reg.score(X_train2, y_train)

x_feature = sm.add_constant(X_train2)
model = sm.OLS(y_train, x_feature)
results_feature = model.fit()
print(results_feature.summary())
pValue = results_feature.pvalues
pValue[pValue<0.05]

0.9115344426897539

                            OLS Regression Results                            
Dep. Variable:              SalePrice   R-squared:                       0.912
Model:                            OLS   Adj. R-squared:                  0.910
Method:                 Least Squares   F-statistic:                     503.6
Date:                Mon, 30 Nov 2020   Prob (F-statistic):               0.00
Time:                        20:55:18   Log-Likelihood:                 1530.8
No. Observations:                2046   AIC:                            -2978.
Df Residuals:                    2004   BIC:                            -2741.
Df Model:                          41                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     6.78

const                     0.000000e+00
GrLivArea                1.105746e-211
Distance                  2.935085e-09
OverallQual               1.330314e-52
OverallCond               1.322262e-68
Fireplaces                3.052968e-09
GarageArea                3.759839e-30
num_bathroom              7.078829e-06
BldgAge                   6.851615e-28
BsmtTtlFinSF              7.406162e-21
Remodeled                 2.853064e-02
ExterQual_num             1.620654e-02
BsmtQual_num              9.507711e-09
KitchenQual_num           4.983658e-06
HeatingQC_num             9.896317e-04
BldgType_Duplex           2.179503e-10
BldgType_Twnhs            3.490368e-18
HouseStyle_1.5Unf         3.455591e-04
HouseStyle_1Story         9.462515e-18
HouseStyle_2Story         1.337532e-03
HouseStyle_SFoyer         3.527103e-04
BsmtExposure_Gd           4.055178e-04
BsmtExposure_Mn           8.874218e-06
BsmtExposure_No           1.084120e-07
CentralAir_Y              4.533229e-03
SaleCondition_Normal     

In [270]:
print(f'Num of features: {len(X_train2.columns)}')
print(f'AIC: {results_feature.aic}')
print(f'BIC: {results_feature.bic}')
print(f'train score: {lin_reg.score(X_train2, y_train)}')
print(f'test score: {lin_reg.score(X_test2, y_test)}')#### train and test error

Num of features: 41
AIC: -2977.637757701563
BIC: -2741.4447959480767
train score: 0.9115344426897539
test score: 0.9188429718714325


#### lasso (use the original features)

In [271]:
# best best fit alpha
reg_lasso_cv = LassoCV(normalize = True, max_iter = 10000)
reg_lasso_cv.fit(X_train, y_train)
reg_lasso_cv.alpha_

LassoCV(alphas=None, copy_X=True, cv=None, eps=0.001, fit_intercept=True,
        max_iter=10000, n_alphas=100, n_jobs=None, normalize=True,
        positive=False, precompute='auto', random_state=None,
        selection='cyclic', tol=0.0001, verbose=False)

3.230835817195771e-05

In [276]:
# put the alpha value back in the original lasso
reg_lasso = Lasso(normalize=True)
reg_lasso.set_params(alpha = reg_lasso_cv.alpha_)
reg_lasso.fit(X_train, y_train)
lasso_coef_df = pd.DataFrame({'feature':X_train.columns,
                              'coef':reg_lasso.coef_})

num_feat = len(lasso_coef_df[lasso_coef_df['coef']!=0])
print(f'{num_feat} non-zero features')
lasso_coef_df[lasso_coef_df['coef']!=0]

Lasso(alpha=3.230835817195771e-05, copy_X=True, fit_intercept=True,
      max_iter=1000, normalize=True, positive=False, precompute=False,
      random_state=None, selection='cyclic', tol=0.0001, warm_start=False)

Lasso(alpha=3.230835817195771e-05, copy_X=True, fit_intercept=True,
      max_iter=1000, normalize=True, positive=False, precompute=False,
      random_state=None, selection='cyclic', tol=0.0001, warm_start=False)

59 non-zero features


,feature,coef
0,GrLivArea,0.493854
1,Distance,-0.017307
2,OverallQual,0.061690
3,OverallCond,0.047575
4,BsmtUnfSF,-0.000496
5,TotRmsAbvGrd,0.001859
6,Fireplaces,0.029912
7,GarageArea,0.000167
9,num_bathroom,0.025387
10,BldgAge,-0.002181


#### test and train errors

In [277]:
print(reg_lasso.score(X_train, y_train))
reg_lasso.score(X_test, y_test)

0.9145479204748681


0.9211465022319905

#### lasso (using engineered features)

In [279]:
reg_lasso_cv = LassoCV(normalize = True, max_iter = 10000)
reg_lasso_cv.fit(X_train2, y_train)
reg_lasso_cv.alpha_

LassoCV(alphas=None, copy_X=True, cv=None, eps=0.001, fit_intercept=True,
        max_iter=10000, n_alphas=100, n_jobs=None, normalize=True,
        positive=False, precompute='auto', random_state=None,
        selection='cyclic', tol=0.0001, verbose=False)

1.9824108989935334e-05

In [280]:
# put the alpha value back in the original lasso
reg_lasso = Lasso(normalize=True)
reg_lasso.set_params(alpha = reg_lasso_cv.alpha_)
reg_lasso.fit(X_train2, y_train)
lasso_coef_df = pd.DataFrame({'feature':X_train2.columns,
                              'coef':reg_lasso.coef_})

num_feat = len(lasso_coef_df[lasso_coef_df['coef']!=0])
print(f'{num_feat} non-zero features')

Lasso(alpha=1.9824108989935334e-05, copy_X=True, fit_intercept=True,
      max_iter=1000, normalize=True, positive=False, precompute=False,
      random_state=None, selection='cyclic', tol=0.0001, warm_start=False)

Lasso(alpha=1.9824108989935334e-05, copy_X=True, fit_intercept=True,
      max_iter=1000, normalize=True, positive=False, precompute=False,
      random_state=None, selection='cyclic', tol=0.0001, warm_start=False)

39 non-zero features


In [281]:
print(reg_lasso.score(X_train2, y_train))
reg_lasso.score(X_test2, y_test)

0.9105871721955869


0.9178068516999432

#### ridge (using original features)

In [282]:
alphas_ridge = np.linspace(1e-5,10,200).tolist()

In [283]:
# best best fit alpha
reg_ridge_cv = RidgeCV(normalize = True, alphas = alphas_ridge)
reg_ridge_cv.fit(X_train, y_train)
reg_ridge_cv.alpha_

RidgeCV(alphas=array([1.00000000e-05, 5.02612060e-02, 1.00512412e-01, 1.50763618e-01,
       2.01014824e-01, 2.51266030e-01, 3.01517236e-01, 3.51768442e-01,
       4.02019648e-01, 4.52270854e-01, 5.02522060e-01, 5.52773266e-01,
       6.03024472e-01, 6.53275678e-01, 7.03526884e-01, 7.53778090e-01,
       8.04029296e-01, 8.54280503e-01, 9.04531709e-01, 9.54782915e-01,
       1.00503412e+00, 1.05528533e+0...
       9.04522709e+00, 9.09547829e+00, 9.14572950e+00, 9.19598070e+00,
       9.24623191e+00, 9.29648312e+00, 9.34673432e+00, 9.39698553e+00,
       9.44723673e+00, 9.49748794e+00, 9.54773915e+00, 9.59799035e+00,
       9.64824156e+00, 9.69849276e+00, 9.74874397e+00, 9.79899518e+00,
       9.84924638e+00, 9.89949759e+00, 9.94974879e+00, 1.00000000e+01]),
        cv=None, fit_intercept=True, gcv_mode=None, normalize=True,
        scoring=None, store_cv_values=False)

1e-05

In [284]:
reg_ridge = Ridge(normalize=True, alpha = reg_ridge_cv.alpha_)
reg_ridge.fit(X_train, y_train)

Ridge(alpha=1e-05, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=True, random_state=None, solver='auto', tol=0.001)

In [286]:
print(reg_ridge.score(X_train, y_train))
reg_ridge.score(X_test, y_test)

0.9168072502749562


0.9220267002749363

#### ridge (using engineered features)

In [287]:
# best best fit alpha
reg_ridge_cv = RidgeCV(normalize = True, alphas = alphas_ridge)
reg_ridge_cv.fit(X_train2, y_train)
reg_ridge_cv.alpha_

RidgeCV(alphas=array([1.00000000e-05, 5.02612060e-02, 1.00512412e-01, 1.50763618e-01,
       2.01014824e-01, 2.51266030e-01, 3.01517236e-01, 3.51768442e-01,
       4.02019648e-01, 4.52270854e-01, 5.02522060e-01, 5.52773266e-01,
       6.03024472e-01, 6.53275678e-01, 7.03526884e-01, 7.53778090e-01,
       8.04029296e-01, 8.54280503e-01, 9.04531709e-01, 9.54782915e-01,
       1.00503412e+00, 1.05528533e+0...
       9.04522709e+00, 9.09547829e+00, 9.14572950e+00, 9.19598070e+00,
       9.24623191e+00, 9.29648312e+00, 9.34673432e+00, 9.39698553e+00,
       9.44723673e+00, 9.49748794e+00, 9.54773915e+00, 9.59799035e+00,
       9.64824156e+00, 9.69849276e+00, 9.74874397e+00, 9.79899518e+00,
       9.84924638e+00, 9.89949759e+00, 9.94974879e+00, 1.00000000e+01]),
        cv=None, fit_intercept=True, gcv_mode=None, normalize=True,
        scoring=None, store_cv_values=False)

1e-05

In [288]:
# put the alpha value back in the original ridge
reg_ridge = Ridge(normalize=True, alpha = reg_ridge_cv.alpha_)
reg_ridge.fit(X_train2, y_train)

Ridge(alpha=1e-05, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=True, random_state=None, solver='auto', tol=0.001)

#### test and train errors

In [289]:
print(reg_ridge.score(X_train2, y_train))
reg_ridge.score(X_test2, y_test)

0.9115344422453068


0.9188429760817837

#### elastic net

In [ ]:
X_train_elnt = X_train_orig.copy()
y_train_elnt = y_train_orig.copy()
X_test_elnt = X_test_orig.copy()
y_test_elnt = y_test_orig.copy()

In [ ]:
reg_elasticnet_cv = ElasticNetCV(l1_ratio = 0.5, cv=5, n_alphas = 200, normalize = True)
reg_elasticnet_cv.fit(X_train_elnt, y_train_elnt)

In [ ]:
print(reg_elasticnet_cv.alpha_)
print(reg_elasticnet_cv.l1_ratio_)

In [ ]:
reg_elasticnet = ElasticNet(alpha = reg_elasticnet_cv.alpha_, normalize = True)
reg_elasticnet.fit(X_train_elnt, y_train_elnt)

In [ ]:
elnt_coef_df = pd.DataFrame({'feature':X_train_elnt.columns,
                              'coef':reg_elasticnet.coef_})

num_feat = len(elnt_coef_df[elnt_coef_df['coef']!=0])
print(f'{num_feat} non-zero features')

In [ ]:
print(reg_elasticnet.score(X_train_elnt, y_train_elnt))
reg_elasticnet.score(X_test_elnt, y_test_elnt)

In [ ]:
ratio = np.linspace(0.01,1,20)
reg_elasticnet_cv = ElasticNetCV(l1_ratio = ratio, cv=5, n_alphas = 200, normalize = True)
reg_elasticnet_cv.fit(X_train_elnt, y_train_elnt)
print(reg_elasticnet_cv.alpha_)
print(reg_elasticnet_cv.l1_ratio_)

#### huber regression

In [ ]:
X_train_huber = X_train.copy()
y_train_huber = y_train.copy()
X_test_huber = X_test.copy()
y_test_huber = y_test.copy()

In [ ]:
reg_huber = HuberRegressor(max_iter = 30000)
reg_huber.fit(X_train_huber, y_train_huber)

In [ ]:
huber_coef_df = pd.DataFrame({'feature':X_train_huber.columns,
                              'coef':reg_huber.coef_})
#huber_coef_df

In [ ]:
print(reg_huber.score(X_train_huber, y_train_huber))
reg_huber.score(X_test_huber, y_test_huber)

#### deceision tree

In [ ]:
from sklearn import ensemble
from sklearn import tree

In [ ]:
tree_model = tree.DecisionTreeRegressor(criterion = 'mse', max_depth = 5)
RF = ensemble.RandomForestRegressor()
BAgg = ensemble.BaggingClassifier()

In [ ]:
X_train_tree = X_train.copy()
y_train_tree = y_train.copy()
X_test_tree = X_test.copy()
y_test_tree = y_test.copy()

In [ ]:
tree_model.set_params()
print(tree_model.fit(X_train_tree, y_train_tree))
print(tree_model.score(X_test_tree, y_test_tree))

In [ ]:
# grid search

#### random forest

In [ ]:
RF.set_params(oob_score=True, random_state=108)
RF.fit(X_train_tree, y_train_tree)

In [ ]:
RF.oob_score_

In [ ]:
forest_grid_params = [{
    "n_estimators": [25, 50, 100],
    "criterion": ['mse','friedman_mse','mae'],
    "min_samples_leaf": range(1, 10),
    "min_samples_split": np.linspace(start=2, stop=30, num=15, dtype=int),
    "random_state": [108]}]
RF_grid = GridSearchCV(RF, forest_grid_params, scoring='r2', cv=5, n_jobs=-1)
%time RF_grid.fit(X_train, y_train)

In [ ]:
# dummify the dataset
orig_dum = pd.get_dummies(ames_df_use, drop_first=True)
print(orig_dum.shape)

# apply log()
orig_dum['SalePrice'] = np.log(orig_dum['SalePrice'])
orig_dum['total_LivArea'] = np.log(orig_dum['total_LivArea'])
orig_dum['total_LivArea'] = np.log(orig_dum['total_LivArea'])
orig_dum['total_LivArea'] = np.log(orig_dum['total_LivArea'])
orig_dum['LotFrontage'] = np.log(orig_dum['LotFrontage']+1)
orig_dum['WoodDeckSF'] = np.log(orig_dum['WoodDeckSF']+1)
orig_dum['OpenPorchSF'] = np.log(orig_dum['OpenPorchSF']+1)
orig_dum['EnclosedPorch'] = np.log(orig_dum['EnclosedPorch']+1)
orig_dum['3SsnPorch'] = np.log(orig_dum['3SsnPorch']+1)
orig_dum['ScreenPorch'] = np.log(orig_dum['ScreenPorch']+1)

In [ ]:
train_orig = orig_dum[orig_dum['YrSold']!=2010]
test_orig = orig_dum[orig_dum['YrSold']==2010]
print(train_orig.shape)
print(test_orig.shape)
X_train_orig, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

X_train_orig = train_orig.drop(['SalePrice','YrSold'],axis=1)
y_train_orig = train_orig['SalePrice']
X_test_orig = test_orig.drop(['SalePrice','YrSold'],axis=1)
y_test_orig = test_orig['SalePrice']

#### initial testing

In [ ]:
lin_reg = LinearRegression().fit(X_train_orig, y_train_orig)
lin_reg.score(X_train_orig, y_train_orig)

x_feature = sm.add_constant(X_train_orig)

model = sm.OLS(y_train_orig, x_feature)
results_feature = model.fit()
print(results_feature.summary())
pValue = results_feature.pvalues
pValue[pValue<0.05]

#X_vif = X_train_dum[pValue[pValue<0.05].index]
X_vif = X_train_orig[pValue[pValue<0.05].drop('const').index]
vif_data = pd.DataFrame() 
vif_data["feature"] = X_vif.columns
vif_data["VIF"] = [variance_inflation_factor(X_vif.values, i) 
                          for i in range(len(X_vif.columns))] 
print(vif_data)

In [ ]:
print(f'Num of features: {len(X_train_orig.columns)}')
print(f'AIC: {results_feature.aic}')
print(f'BIC: {results_feature.bic}')
print(f'train score: {lin_reg.score(X_train_orig, y_train_orig)}')
print(f'test score: {lin_reg.score(X_test_orig, y_test_orig)}')

#### minimum sample/class size

In [ ]:
(1.96 * statistics.stdev(ames_df_use['SalePrice']) / 20000)**2

In [ ]:
set(ames_df['Neighborhood'])

In [150]:
for col in ames_df.columns.tolist():
    if col.find('Bsmt')!=-1:
        print(col)

BsmtQual
BsmtCond
BsmtExposure
BsmtFinType1
BsmtFinSF1
BsmtFinType2
BsmtFinSF2
BsmtUnfSF
TotalBsmtSF
BsmtFullBath
BsmtHalfBath
BsmtHalfBath2
TtlBsmtSF


In [ ]:
# Decision Tree Model
tree_reg = DecisionTreeRegressor(max_depth=10).fit(X_train1,y_train1)
print(f'R^2 of Train set: {tree_reg.score(X_train1,y_train1)}')
print(f'R^2 Test set: {tree_reg.score(X_test1,y_test1)}')



In [ ]:
# Random Forest Model
forest_reg = RandomForestRegressor(n_estimators=100,max_features=5).fit(X_train1,y_train1)
print(f'R^2 of Train set: {forest_reg.score(X_train1,y_train1)}')
print(f'R^2 Test set: {forest_reg.score(X_test1,y_test1)}')

In [ ]:
# from sklearn.metrics import mean_squared_error
# from sklearn.metrics import r2_score
# housing_prediction = lin_reg.predict(X_test)
# mean_squared_error(y_train,y_test)
# lin_reg2 = LinearRegression().fit(X_train,y_train)
# lin_reg.r2_score